# 🏭 Theology AI Lab - Cloud Indexer

이 노트북은 **Google Drive**에 저장된 신학 문서(PDF/EPUB)를 **T4 GPU**를 사용하여 고속으로 인덱싱합니다.

### 🚀 사용법
1. **`01_Library/inbox`** 폴더에 PDF 파일을 넣으세요.
2. 상단 메뉴의 **`런타임 > 모두 실행`**을 클릭하세요.
3. 완료되면 로컬 Streamlit 앱에서 검색하세요.

### 📂 필요한 폴더 구조 (구글 드라이브)
```
내 드라이브/Theology_AI_LAB/
├── 01_Library/inbox/     ← PDF 파일을 여기에 넣으세요 (하위 폴더 지원)
├── 01_Library/archive/   ← 처리 완료된 파일
└── 02_Brain/vector_db/   ← 자동 생성됨
```

In [ ]:
# 1️⃣ 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')
print('✅ 드라이브 연결 완료!')

In [ ]:
# 2️⃣ 필수 라이브러리 설치 (약 1분 소요)
print('📦 라이브러리 설치 중...')
!pip install -q langchain langchain-community langchain-chroma chromadb sentence-transformers pymupdf pytesseract rank_bm25 ebooklib beautifulsoup4
!apt-get install -y tesseract-ocr > /dev/null 2>&1
print('✅ 설치 완료!')

In [ ]:
# 3️⃣ GitHub에서 최신 인덱서 코드 다운로드
import os
from pathlib import Path

GITHUB_RAW_BASE = 'https://raw.githubusercontent.com/mitmirsein/Theology_Ai_Lab/main/03_System'
WORK_DIR = Path('/content/theology_indexer')
WORK_DIR.mkdir(exist_ok=True)

# 필요한 파일들 다운로드
files_to_download = [
    'processor_v4.py',
    'pipeline/__init__.py',
    'pipeline/embedder.py',
    'pipeline/metadata_parser.py',
    'pipeline/semantic_chunker.py',
    'pipeline/router.py',
    'pipeline/chunker.py',
    'pipeline/searcher.py',
    'utils/__init__.py',
    'utils/ocr_pdf_processor.py',
    'utils/page_number_detector.py',
]

print('📥 GitHub에서 최신 코드 다운로드 중...')
for f in files_to_download:
    url = f'{GITHUB_RAW_BASE}/{f}'
    local_path = WORK_DIR / f
    # 부모 디렉토리가 없으면 생성
    local_path.parent.mkdir(parents=True, exist_ok=True)
    !wget -q -O "{local_path}" "{url}" 2>/dev/null || echo "⚠️ {f} 다운로드 실패"

# Python 경로에 추가
import sys
if str(WORK_DIR) not in sys.path:
    sys.path.insert(0, str(WORK_DIR))

print('✅ 코드 준비 완료!')

In [ ]:
# 4️⃣ 경로 설정
PROJECT_ROOT = Path('/content/drive/MyDrive/Theology_AI_LAB')

inbox_dir = PROJECT_ROOT / '01_Library/inbox'
archive_dir = PROJECT_ROOT / '01_Library/archive'
db_dir = PROJECT_ROOT / '02_Brain/vector_db'

# 폴더 생성 (없으면)
inbox_dir.mkdir(parents=True, exist_ok=True)
archive_dir.mkdir(parents=True, exist_ok=True)
db_dir.mkdir(parents=True, exist_ok=True)

print(f'📂 Inbox: {inbox_dir}')
print(f'📦 Archive: {archive_dir}')
print(f'🧠 Vector DB: {db_dir}')

# Inbox 파일 확인 (하위 폴더 포함 재귀 검색 rglob 사용)
inbox_files = list(inbox_dir.rglob('*.pdf')) + list(inbox_dir.rglob('*.epub')) + list(inbox_dir.rglob('*.txt'))
print(f'\n📄 처리 대기 파일: {len(inbox_files)}개')
for f in inbox_files[:5]:
    print(f'   - {f.name}')
if len(inbox_files) > 5:
    print(f'   ... 외 {len(inbox_files)-5}개')

In [ ]:
# 5️⃣ 인덱싱 실행
if len(inbox_files) == 0:
    print('⚠️ Inbox에 파일이 없습니다!')
    print('💡 구글 드라이브의 Theology_AI_LAB/01_Library/inbox/ 폴더(또는 하위 폴더)에 PDF 파일을 넣어주세요.')
else:
    print('🚀 인덱싱을 시작합니다...\n')
    
    from processor_v4 import ProjectV4Processor
    
    processor = ProjectV4Processor(db_dir=str(db_dir))
    
    # processor 내부에서도 rglob을 사용하므로 하위 폴더 파일도 잘 처리됩니다.
    for update in processor.index_files_with_progress(inbox_dir, archive_dir):
        status = update.get('status')
        msg = update.get('message')
        
        if status == 'indexing':
            print(f'📖 {msg}')
        elif status == 'done':
            print(f'\n🎉 {msg}')
            stats = update.get('session_stats', {})
            print(f'   - 처리된 파일: {stats.get("processed_files", 0)}')
            print(f'   - 총 청크: {stats.get("total_chunks", 0)}')
        elif status == 'error':
            print(f'❌ 오류: {msg}')
    
    print('\n' + '='*50)
    print('✅ 완료! 로컬 앱에서 검색해보세요.')
    print('='*50)